[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioImagingUK/Training/blob/main/notebooks/Read_Image_Using_Python.ipynb)
## Learning Objectives
* How to read various image formats using different libraries.

The reading functions of most Python library will return by default a [Numpy array](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

### imageio

When working with images in Python a library to consider is [scikit-image](https://scikit-image.org/) library. This library is useful when working with TIFF images. But, supporting a format does not necessary mean than all images in that format will be correctly read.
Another library to explore is [imageio](https://imageio.readthedocs.io/en/stable/). Scipy has deprecated their image I/O functionality for [imageio](https://imageio.readthedocs.io/en/stable/user_guide/scipy.html).

The `imageio` library supports reading local data or remote data

In [ ]:
# Download a file in the Colab environment so we can test the local reading
!wget https://github.com/imageio/imageio-binaries/raw/master/images/bacterial_colony.tif

In [ ]:
path = "/content/bacterial_colony.tif"
import imageio.v3 as iio
img = iio.imread(path)

In [ ]:
print(img.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img[0, :, :])

In [ ]:
path = "https://downloads.openmicroscopy.org/images/TIFF/condensation/A1.pattern1.tif"

In [ ]:
img = iio.imread(path)
print(img.shape)

To understand the image e.g. what are the dimensions, it is important to be able to read the metadata.
This information might not always be available in the image file itself.

In [ ]:
metadata = iio.immeta(path)
print(metadata.get('Info'))

In [ ]:
from ipywidgets import *

def update(t=0, c=0):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(img[t, c, :, :])
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, t=widgets.IntSlider(value=0, min=0, max=img.shape[0]-1, step=1, description="Select T", continuous_update=False),
        c=widgets.IntSlider(value=0, min=0, max=img.shape[1]-1, step=1, description="Select C", continuous_update=False))

### tifffile
[tifffile](https://github.com/cgohlke/tifffile) is the library for working with TIFF data.

In [ ]:
!pip install -U tifffile[all]

In [ ]:
import tifffile as tff
image = tff.imread('/content/bacterial_colony.tif')

In [ ]:
plt.imshow(image[0, :, :])

### BioIO

In order to analyse your image data, you need to be able to read the numerous proprietary file formats (PFFs). Bio-Formats is the most comprehensive tool in Java to read PFFs, but no such library exists in Python yet.

Nonetheless the [BioIO](https://github.com/bioio-devs/bioio) library is a more domain-specific Python library for accessing data. It wraps several libraries for microscopy vendor's formats (including ``nd2``, ``czi``) and has a Python wrapper around the reference Java library for reading many proprietary file formats [Bio-Formats](https://www.openmicroscopy.org/bio-formats/). This is obviously not a pure Python solution since it requires Java installed in order to run. The library also offers the ability to convert to OME-TIFF.

In [ ]:
!wget https://downloads.openmicroscopy.org/images/ND2/aryeh/MeOh_high_fluo_011.nd2

In [ ]:
%pip install bioio
%pip install bioio-nd2

# dependency required to save as OME-TIFF%pip install bioio-ome-tiff

In [ ]:
from bioio import BioImage
import bioio_nd2

img = BioImage("/content/MeOh_high_fluo_011.nd2", reader=bioio_nd2.Reader)

img.dims.order # Return the dimension order. Note: This information is not available when using scikit-image only.

img.dims.X
img.channel_names
img.physical_pixel_sizes.X
img.shape
img.data

# Return 4D CZYX numpy array
data = img.get_image_data("CZYX", T=0)
data.shape

# Return 3D ZYX numpy array
data = img.get_image_data("ZYX", T=0, C=0)
print(data.shape)

# Get the id of the current operating scene. In our case we only have one scene
img.current_scene

# Get a list valid scene ids
img.scenes

Save the image as OME-TIFF

In [ ]:
# Save the image as OME-TIFF
from bioio_ome_tiff.writers import OmeTiffWriter
OmeTiffWriter.save(data, "/content/MeOh_high_fluo_011.nd2.ome.tiff")

### License (BSD 2-Clause)
Copyright (C) 2026 BioImagingUK. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.